In [1]:
import pandas as pd

In [2]:
# my index column or the first column is unnamed
df = pd.read_csv("./data/progressive_moves_500.csv", index_col = 0)
df.head()

,opening_name,prev_moves,last_move
0,Slav Defense: Exchange Variation,NaN,d4
1,Slav Defense: Exchange Variation,d4,d5
2,Slav Defense: Exchange Variation,d4 d5,c4
3,Slav Defense: Exchange Variation,d4 d5 c4,c6
4,Slav Defense: Exchange Variation,d4 d5 c4 c6,cxd5


# Loading API keys

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
openai_api_key = os.getenv("openai_api_key")

## Setting up openai api call environment

In [5]:
from openai import OpenAI
client = OpenAI(api_key = openai_api_key)

In [6]:
df.iloc[28]

opening_name               Nimzowitsch Defense: Kennedy Variation
prev_moves      d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...
last_move                                                    Bb4+
Name: 28, dtype: object

In [7]:
prev_moves, last_move = df.iloc[28]["prev_moves"], df.iloc[28]["last_move"]
prev_moves, last_move

('d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6 Qe5+ Nxe5 c4', 'Bb4+')

In [8]:
system_content_prim = '''You will be provided with a list of chess moves in Algebraic Notation,
            and your task is to analyse all previous moves and explain why last move was taken.'''

In [9]:
recursion_psuedo_code = '''
function explainMoves(moves):
    # base case
    if len(moves) == 1:
        return generateExplanation(moves[0])
    
    # recursion
    previous_explanation = explainMoves(moves[:-1])
    
    # generate explanation using returned explanation and last move in the list
    new_explanation = generateNewExplanation(previous_explanation, moves[-1])
    
    return new_explanation

function generateExplanation(move):
    return explain_the_move-{move}

function generateNewExplanation(previous_explanation, move):
    return explain_the_move_{move}_based_on_previous_explanation

'''

In [11]:
system_content_recur = f'''Assume you are a chess master.
            You will be provided with a list of chess moves in Algebraic Notation,
            and your task is to recursively analyse each previous move.
            A pseudo code for such recursive function is provided here - {recursion_psuedo_code}'''

In [12]:
user_content = f"previous moves are {prev_moves}, analyse each move and explain why the last move {last_move} was taken."

- wrong explanation dxe5: This move is a capture of the f6 pawn, gaining material advantage.
- from explanation 9, everything is wrong cause it escaped the repeat fxe5


In [15]:
stream = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    # messages=[{"role": "user", "content": setup}],
    messages = [
    {
      "role": "system",
      "content": system_content_recur
    },
    {
      "role": "user",
      "content": user_content
    }
  ],
    stream=True,
)
print()
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")


Here is the recursive implementation in pseudo code to analyse each move:

```
function explainMoves(moves):
    # base case
    if len(moves) == 1:
        return generateExplanation(moves[0])
    
    # recursion
    previous_explanation = explainMoves(moves[:-1])
    
    # generate explanation using returned explanation and last move in the list
    new_explanation = generateNewExplanation(previous_explanation, moves[-1])
    
    return new_explanation

function generateExplanation(move):
    return explain_the_move_{move}

function generateNewExplanation(previous_explanation, move):
    return explain_the_move_{move}_based_on_previous_explanation
```

To breakdown each move:

1. d4: This move is the Queen's Pawn Opening. It controls the central squares and prepares for development.
2. Nc6: This move is the Knight development and prepares for central control. It also prevents White from playing e4.
3. e4: This move is the King's Pawn Opening, controlling the center and preparing 

# Testing the prim prompt

In [17]:
stream = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    # messages=[{"role": "user", "content": setup}],
    messages = [
    {
      "role": "system",
      "content": system_content_prim
    },
    {
      "role": "user",
      "content": user_content
    }
  ],
    stream=True,
)
print()
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")


1. d4 - This move opens up the center and allows White to control the central squares d4 and e4. It also prepares for development of White's pieces.

2. Nc6 - Black develops the knight to a central square, preparing for further development and control of the center.

3. e4 - White expands in the center and continues development of their pieces.

4. e5 - Black counters in the center by challenging White's central pawn and gaining space.

5. f4 - White continues to push forward in the center, gaining more space and threatening Black's e5 pawn.

6. f6 - Black defends the e5 pawn and gains control over the central square on f6.

7. dxe5 - White captures the e5 pawn and gains material advantage.

8. fxe5 - Black recaptures the pawn on e5 to regain material equality.

9. Nxe5 - White attacks the knight on e5 and gains a tempo.

10. Qd4 - White brings the queen out, attacking the knight on c6 and preparing to castle kingside.

11. Nc6 - Black defends the attacked knight on c6 and also attack

# for loop explicit chain of thougt prompting

In [45]:
def gen_cot_prompt(prev_moves : str, last_move : str) -> str:
    prev_moves = (prev_moves+f" {last_move}").split(" ")
    cot_moves = [f"White: {prev_moves[i]}, Black: {prev_moves[i+1]}" if i + 1 < len(prev_moves) else f"White: {prev_moves[i]}" for i in range(0, len(prev_moves), 2)]
    return cot_moves

In [46]:
print("\n".join(gen_cot_prompt(prev_moves, last_move)))

White: d4, Black: Nc6
White: e4, Black: e5
White: f4, Black: f6
White: dxe5, Black: fxe5
White: fxe5, Black: Nxe5
White: Qd4, Black: Nc6
White: Qe5+, Black: Nxe5
White: c4, Black: Bb4+


In [48]:
system_content_cot = '''Assume you are a chess master.
You will be provided with a list of chess move pairs in Algebraic Notation where 1st move is by White and 2nd by Black.
Your task is to analyse each pair and then generate the rationale behind the last move.
'''
user_content_cot = f"Previous move pairs are - {gen_cot_prompt(prev_moves, last_move)}, last move is {last_move}"

In [49]:
stream = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    # messages=[{"role": "user", "content": setup}],
    messages = [
    {
      "role": "system",
      "content": system_content_cot
    },
    {
      "role": "user",
      "content": user_content_cot
    }
  ],
    stream=True,
)
print()
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")


In the given chess game, the last move played was Black's Bishop moving to b4+ (Bb4+). 

Analyzing the previous moves:
1. White played d4, which is a common opening move controlling the center and allowing the queen and bishop to develop.
2. Black responded with Nc6, developing the knight and preparing for further development. 
3. White played e4, reinforcing the center and preparing for the development of the bishop and queen.
4. Black mirrored White's move with e5, also controlling the center. 
5. White played f4, advancing the pawn to support the center and potentially preparing for a kingside attack.
6. Black responded with f6, challenging White's pawn and preparing to advance in the center.
7. White captured Black's pawn with dxe5, gaining material advantage.
8. Black captured White's pawn with fxe5, regaining the material.
9. White captured Black's knight with fxe5, aiming to maintain the material advantage. 
10. Black moved the knight to c6, attacking White's queen and preparin